In [1]:
!pip install transformers datasets torch scikit-learn pandas numpy matplotlib seaborn accelerate

In [2]:
import numpy as np
from huggingface_hub import login
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.model_selection import KFold
import torch
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata
hf_token = userdata.get('huggingface')
login(hf_token)

In [3]:
np.random.seed(42)
torch.manual_seed(42)

In [4]:
dataset = load_dataset("Zlovoblachko/DeepSeek_dim3_balanced")

README.md:   0%|          | 0.00/798 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1392 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/174 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/175 [00:00<?, ? examples/s]

In [5]:
print("Dataset splits:")
print(dataset)
print("\nSample from training set:")
print(dataset['train'][0])
print("\nColumn names:")
print(dataset['train'].column_names)

Dataset splits:
DatasetDict({
    train: Dataset({
        features: ['Previous clause', 'Previous clause label', 'Target clause', 'Target clause label', 'Number of the target clause in a sentence', 'Annotation', 'Label', '__index_level_0__'],
        num_rows: 1392
    })
    dev: Dataset({
        features: ['Previous clause', 'Previous clause label', 'Target clause', 'Target clause label', 'Number of the target clause in a sentence', 'Annotation', 'Label', '__index_level_0__'],
        num_rows: 174
    })
    test: Dataset({
        features: ['Previous clause', 'Previous clause label', 'Target clause', 'Target clause label', 'Number of the target clause in a sentence', 'Annotation', 'Label', '__index_level_0__'],
        num_rows: 175
    })
})

Sample from training set:
{'Previous clause': "('that is a waste of time', 'acl')", 'Previous clause label': 'Thesis', 'Target clause': "('because most people tend', 'advcl')", 'Target clause label': 'Thesis', 'Number of the target clause 

In [6]:
train_labels = [item['Label'] for item in dataset['train']]
val_labels = [item['Label'] for item in dataset['dev']]
test_labels = [item['Label'] for item in dataset['test']]

In [7]:
print(f"Training set - H: {train_labels.count('H')}, L: {train_labels.count('L')}")
print(f"Validation set - H: {val_labels.count('H')}, L: {val_labels.count('L')}")
print(f"Test set - H: {test_labels.count('H')}, L: {test_labels.count('L')}")

Training set - H: 480, L: 432
Validation set - H: 60, L: 54
Test set - H: 60, L: 55


In [8]:
column_combinations = {
    'target_only': ['Target clause'],
    'target_previous': ['Target clause', 'Previous clause'],
    'target_previous_label': ['Target clause', 'Previous clause', 'Previous clause label'],
    'target_with_position': ['Target clause', 'Number of the target clause in a sentence'],
    'all_columns': ['Target clause', 'Previous clause', 'Previous clause label', 'Number of the target clause in a sentence']
}

In [9]:
print("Column combinations for ablation study:")
for name, cols in column_combinations.items():
    print(f"{name}: {cols}")

Column combinations for ablation study:
target_only: ['Target clause']
target_previous: ['Target clause', 'Previous clause']
target_previous_label: ['Target clause', 'Previous clause', 'Previous clause label']
target_with_position: ['Target clause', 'Number of the target clause in a sentence']
all_columns: ['Target clause', 'Previous clause', 'Previous clause label', 'Number of the target clause in a sentence']


In [10]:
def concatenate_columns(example, columns):
    """Concatenate specified columns with [SEP] token"""
    texts = []
    for col in columns:
        if example[col] and str(example[col]).strip():
            texts.append(str(example[col]).strip())
    return ' [SEP] '.join(texts)

In [11]:
def prepare_dataset(dataset_split, columns, tokenizer, max_length=512):
    """Tokenize and prepare dataset"""
    texts = [concatenate_columns(example, columns) for example in dataset_split]
    labels = [1 if example['Label'] == 'H' else 0 for example in dataset_split]

    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels)
    }

In [12]:
class TextDataset(torch.utils.data.Dataset):
    """PyTorch Dataset wrapper"""
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.encodings['labels'][idx]
        }

    def __len__(self):
        return len(self.encodings['labels'])

In [13]:
def compute_metrics(pred):
    """Compute evaluation metrics"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [14]:
def cross_validate_model(model_name, train_dataset, columns, n_folds=5):
    """Perform k-fold cross-validation and return metrics with CI"""
    print(f"\n{'='*60}")
    print(f"Cross-validating {model_name} with columns: {columns}")
    print(f"{'='*60}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare data
    texts = [concatenate_columns(example, columns) for example in train_dataset]
    labels = np.array([1 if example['Label'] == 'H' else 0 for example in train_dataset])

    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    fold_metrics = {
        'accuracy': [],
        'f1': [],
        'precision': [],
        'recall': []
    }

    for fold, (train_idx, val_idx) in enumerate(kfold.split(texts)):
        print(f"\nFold {fold + 1}/{n_folds}")

        # Split data
        train_texts = [texts[i] for i in train_idx]
        train_labels = labels[train_idx]
        val_texts = [texts[i] for i in val_idx]
        val_labels = labels[val_idx]

        # Tokenize
        train_encodings = tokenizer(train_texts, truncation=True, padding='max_length',
                                    max_length=512, return_tensors='pt')
        val_encodings = tokenizer(val_texts, truncation=True, padding='max_length',
                                  max_length=512, return_tensors='pt')

        train_data = TextDataset({
            'input_ids': train_encodings['input_ids'],
            'attention_mask': train_encodings['attention_mask'],
            'labels': torch.tensor(train_labels)
        })

        val_data = TextDataset({
            'input_ids': val_encodings['input_ids'],
            'attention_mask': val_encodings['attention_mask'],
            'labels': torch.tensor(val_labels)
        })

        # Initialize model
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2
        )

        # Training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_{model_name.split("/")[-1]}_fold{fold}',
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=16,
            learning_rate=2e-5,
            weight_decay=0.01,
            eval_strategy='epoch',
            save_strategy='epoch',
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            logging_dir='./logs',
            logging_steps=10,
            seed=42
        )

        # Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=val_data,
            compute_metrics=compute_metrics
        )

        # Train
        trainer.train()

        # Evaluate
        results = trainer.evaluate()

        fold_metrics['accuracy'].append(results['eval_accuracy'])
        fold_metrics['f1'].append(results['eval_f1'])
        fold_metrics['precision'].append(results['eval_precision'])
        fold_metrics['recall'].append(results['eval_recall'])

        print(f"Fold {fold + 1} Results:")
        print(f"  Accuracy: {results['eval_accuracy']:.4f}")
        print(f"  F1: {results['eval_f1']:.4f}")
        print(f"  Precision: {results['eval_precision']:.4f}")
        print(f"  Recall: {results['eval_recall']:.4f}")

    # Calculate statistics
    stats = {}
    for metric, values in fold_metrics.items():
        values_array = np.array(values)
        mean = values_array.mean()
        std = values_array.std()
        # 95% confidence interval
        ci_lower = mean - 1.96 * (std / np.sqrt(n_folds))
        ci_upper = mean + 1.96 * (std / np.sqrt(n_folds))

        stats[metric] = {
            'mean': mean,
            'std': std,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'all_folds': values
        }

    print(f"\n{'='*60}")
    print(f"Cross-Validation Summary for {model_name}")
    print(f"{'='*60}")
    for metric, stat in stats.items():
        print(f"{metric.capitalize()}:")
        print(f"  Mean: {stat['mean']:.4f}")
        print(f"  Std: {stat['std']:.4f}")
        print(f"  95% CI: [{stat['ci_lower']:.4f}, {stat['ci_upper']:.4f}]")

    return stats, fold_metrics

In [15]:
def train_and_evaluate_best_model(model_name, train_dataset, val_dataset, test_dataset, columns):
    """Train model on full training set and evaluate on test set"""
    print(f"\n{'='*60}")
    print(f"Training {model_name} on full training set")
    print(f"Columns: {columns}")
    print(f"{'='*60}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare datasets
    train_data = prepare_dataset(train_dataset, columns, tokenizer)
    val_data = prepare_dataset(val_dataset, columns, tokenizer)
    test_data = prepare_dataset(test_dataset, columns, tokenizer)

    train_dataset_obj = TextDataset(train_data)
    val_dataset_obj = TextDataset(val_data)
    test_dataset_obj = TextDataset(test_data)

    # Initialize model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{model_name.split("/")[-1]}_best',
        num_train_epochs=5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        logging_dir='./logs',
        logging_steps=10,
        seed=42
    )

    # Trainer with early stopping
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_obj,
        eval_dataset=val_dataset_obj,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    # Train
    trainer.train()

    # Evaluate on test set
    test_results = trainer.evaluate(test_dataset_obj)

    # Get predictions for detailed analysis
    predictions = trainer.predict(test_dataset_obj)
    pred_labels = predictions.predictions.argmax(-1)
    true_labels = test_data['labels'].numpy()

    print(f"\n{'='*60}")
    print(f"Test Set Results for {model_name}")
    print(f"{'='*60}")
    print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
    print(f"F1: {test_results['eval_f1']:.4f}")
    print(f"Precision: {test_results['eval_precision']:.4f}")
    print(f"Recall: {test_results['eval_recall']:.4f}")

    # Classification report
    print("\nDetailed Classification Report:")
    print(classification_report(true_labels, pred_labels, target_names=['L', 'H']))

    # Confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)

    return {
        'test_results': test_results,
        'predictions': pred_labels,
        'true_labels': true_labels,
        'confusion_matrix': cm,
        'model': model,
        'trainer': trainer
    }

In [16]:
model_names = [
    'bert-base-uncased',
    'roberta-base',
    'answerdotai/ModernBERT-base'
]

In [17]:
cv_results = {}

In [18]:
selected_columns = column_combinations['target_only']

In [19]:
for model_name in model_names:
    try:
        stats, fold_metrics = cross_validate_model(
            model_name,
            dataset['train'],
            selected_columns,
            n_folds=3
        )
        cv_results[model_name] = {
            'stats': stats,
            'fold_metrics': fold_metrics
        }
    except Exception as e:
        print(f"Error with {model_name}: {str(e)}")
        continue


Cross-validating bert-base-uncased with columns: ['Target clause']


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Fold 2 Results:
  Accuracy: 0.7500
  F1: 0.7466
  Precision: 0.7456
  Recall: 0.7500

Fold 3/3


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.514800,0.439195,0.780172,0.781068,0.782232,0.780172
2,0.341800,0.445464,0.793103,0.794208,0.795797,0.793103


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.514800,0.439195,0.780172,0.781068,0.782232,0.780172
2,0.341800,0.445464,0.793103,0.794208,0.795797,0.793103
3,0.311000,0.538013,0.812500,0.806725,0.810352,0.812500


Fold 3 Results:
  Accuracy: 0.8125
  F1: 0.8067
  Precision: 0.8104
  Recall: 0.8125

Cross-Validation Summary for answerdotai/ModernBERT-base
Accuracy:
  Mean: 0.8010
  Std: 0.0378
  95% CI: [0.7582, 0.8438]
F1:
  Mean: 0.7964
  Std: 0.0371
  95% CI: [0.7544, 0.8383]
Precision:
  Mean: 0.7980
  Std: 0.0388
  95% CI: [0.7542, 0.8419]
Recall:
  Mean: 0.8010
  Std: 0.0378
  95% CI: [0.7582, 0.8438]


In [20]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
metrics = ['accuracy', 'f1', 'precision', 'recall']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]

    model_names_list = list(cv_results.keys())
    means = [cv_results[m]['stats'][metric]['mean'] for m in model_names_list]
    stds = [cv_results[m]['stats'][metric]['std'] for m in model_names_list]

    x_pos = np.arange(len(model_names_list))
    ax.bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7)
    ax.set_xlabel('Model')
    ax.set_ylabel(metric.capitalize())
    ax.set_title(f'{metric.capitalize()} across Models (with Std Dev)')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([m.split('/')[-1] for m in model_names_list], rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('cv_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [21]:
summary_data = []
for model_name, results in cv_results.items():
    for metric in ['accuracy', 'f1', 'precision', 'recall']:
        stats = results['stats'][metric]
        summary_data.append({
            'Model': model_name.split('/')[-1],
            'Metric': metric.capitalize(),
            'Mean': f"{stats['mean']:.4f}",
            'Std': f"{stats['std']:.4f}",
            '95% CI': f"[{stats['ci_lower']:.4f}, {stats['ci_upper']:.4f}]"
        })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("CROSS-VALIDATION SUMMARY TABLE")
print("="*80)
print(summary_df.to_string(index=False))


CROSS-VALIDATION SUMMARY TABLE
            Model    Metric   Mean    Std           95% CI
bert-base-uncased  Accuracy 0.7967 0.0148 [0.7800, 0.8134]
bert-base-uncased        F1 0.7930 0.0143 [0.7768, 0.8091]
bert-base-uncased Precision 0.7929 0.0147 [0.7763, 0.8095]
bert-base-uncased    Recall 0.7967 0.0148 [0.7800, 0.8134]
     roberta-base  Accuracy 0.7960 0.0159 [0.7780, 0.8139]
     roberta-base        F1 0.7922 0.0149 [0.7754, 0.8091]
     roberta-base Precision 0.7923 0.0156 [0.7746, 0.8099]
     roberta-base    Recall 0.7960 0.0159 [0.7780, 0.8139]
  ModernBERT-base  Accuracy 0.8010 0.0378 [0.7582, 0.8438]
  ModernBERT-base        F1 0.7964 0.0371 [0.7544, 0.8383]
  ModernBERT-base Precision 0.7980 0.0388 [0.7542, 0.8419]
  ModernBERT-base    Recall 0.8010 0.0378 [0.7582, 0.8438]


In [22]:
best_model_name = max(cv_results.keys(),
                      key=lambda x: cv_results[x]['stats']['f1']['mean'])
print(f"\nBest performing model: {best_model_name}")
print(f"Mean F1 score: {cv_results[best_model_name]['stats']['f1']['mean']:.4f}")


Best performing model: answerdotai/ModernBERT-base
Mean F1 score: 0.7964


In [23]:
best_model_results = train_and_evaluate_best_model(
    best_model_name,
    dataset['train'],
    dataset['dev'],
    dataset['test'],
    selected_columns
)


Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.462900,0.490517,0.793103,0.780756,0.794133,0.793103
2,0.262600,0.659473,0.712644,0.719184,0.780411,0.712644


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.462900,0.490517,0.793103,0.780756,0.794133,0.793103
2,0.262600,0.659473,0.712644,0.719184,0.780411,0.712644
3,0.460100,0.715769,0.810345,0.801274,0.810813,0.810345
4,0.176000,0.849862,0.804598,0.804598,0.804598,0.804598
5,0.119300,0.926008,0.821839,0.820712,0.820110,0.821839



Test Set Results for answerdotai/ModernBERT-base
Accuracy: 0.7714
F1: 0.7731
Precision: 0.7756
Recall: 0.7714

Detailed Classification Report:
              precision    recall  f1-score   support

           L       0.84      0.81      0.82       115
           H       0.66      0.70      0.68        60

    accuracy                           0.77       175
   macro avg       0.75      0.75      0.75       175
weighted avg       0.78      0.77      0.77       175



In [24]:
best_model_results = train_and_evaluate_best_model(
    best_model_name,
    dataset['train'],
    dataset['dev'],
    dataset['test'],
    selected_columns
)


Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.460200,0.437507,0.798851,0.792056,0.795484,0.798851
2,0.239700,0.508271,0.793103,0.795807,0.801906,0.793103


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.460200,0.437507,0.798851,0.792056,0.795484,0.798851
2,0.239700,0.508271,0.793103,0.795807,0.801906,0.793103
3,0.468200,0.775044,0.775862,0.774444,0.773512,0.775862
4,0.097200,1.035452,0.827586,0.827586,0.827586,0.827586
5,0.048500,1.178405,0.821839,0.820712,0.820110,0.821839



Test Set Results for answerdotai/ModernBERT-base
Accuracy: 0.7600
F1: 0.7617
Precision: 0.7643
Recall: 0.7600

Detailed Classification Report:
              precision    recall  f1-score   support

           L       0.83      0.80      0.81       115
           H       0.64      0.68      0.66        60

    accuracy                           0.76       175
   macro avg       0.73      0.74      0.74       175
weighted avg       0.76      0.76      0.76       175



In [25]:
best_model_results = train_and_evaluate_best_model(
    best_model_name,
    dataset['train'],
    dataset['dev'],
    dataset['test'],
    selected_columns
)


Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.445700,0.445878,0.810345,0.799800,0.813048,0.810345
2,0.280000,0.498890,0.804598,0.807151,0.813158,0.804598


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.445700,0.445878,0.810345,0.799800,0.813048,0.810345
2,0.280000,0.498890,0.804598,0.807151,0.813158,0.804598
3,0.393700,0.712485,0.781609,0.779727,0.778722,0.781609
4,0.090500,1.055713,0.839080,0.839080,0.839080,0.839080
5,0.052100,1.181593,0.839080,0.837693,0.837271,0.839080



Test Set Results for answerdotai/ModernBERT-base
Accuracy: 0.7429
F1: 0.7459
Precision: 0.7513
Recall: 0.7429

Detailed Classification Report:
              precision    recall  f1-score   support

           L       0.82      0.77      0.80       115
           H       0.61      0.68      0.65        60

    accuracy                           0.74       175
   macro avg       0.72      0.73      0.72       175
weighted avg       0.75      0.74      0.75       175



In [30]:
plt.figure(figsize=(8, 6))
cm = best_model_results['confusion_matrix']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['L', 'H'], yticklabels=['L', 'H'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title(f'Confusion Matrix - {best_model_name.split("/")[-1]}')
plt.savefig('confusion_matrix_best_model.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("\n" + "="*80)
print("ABLATION STUDY: COLUMN COMBINATIONS")
print("="*80)

ablation_results = {}

for combo_name, columns in column_combinations.items():
    print(f"\nTesting combination: {combo_name}")
    print(f"Columns: {columns}")

    try:
        results = train_and_evaluate_best_model(
            best_model_name,
            dataset['train'],
            dataset['dev'],
            dataset['test'],
            columns
        )

        ablation_results[combo_name] = {
            'columns': columns,
            'accuracy': results['test_results']['eval_accuracy'],
            'f1': results['test_results']['eval_f1'],
            'precision': results['test_results']['eval_precision'],
            'recall': results['test_results']['eval_recall']
        }
    except Exception as e:
        print(f"Error with {combo_name}: {str(e)}")
        continue


ABLATION STUDY: COLUMN COMBINATIONS

Testing combination: target_only
Columns: ['Target clause']

Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.445700,0.445876,0.810345,0.799800,0.813048,0.810345
2,0.265200,0.499848,0.793103,0.792250,0.791617,0.793103
3,0.389600,0.787858,0.798851,0.797578,0.796811,0.798851



Test Set Results for answerdotai/ModernBERT-base
Accuracy: 0.7429
F1: 0.7285
Precision: 0.7344
Recall: 0.7429

Detailed Classification Report:
              precision    recall  f1-score   support

           L       0.76      0.89      0.82       115
           H       0.68      0.47      0.55        60

    accuracy                           0.74       175
   macro avg       0.72      0.68      0.69       175
weighted avg       0.73      0.74      0.73       175


Testing combination: target_previous
Columns: ['Target clause', 'Previous clause']

Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause', 'Previous clause']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.351700,0.472086,0.798851,0.785999,0.802220,0.798851


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.351700,0.472086,0.798851,0.785999,0.802220,0.798851
2,0.410400,0.426191,0.833333,0.836440,0.848908,0.833333
3,0.285400,0.655760,0.827586,0.815820,0.838449,0.827586
4,0.137300,0.690411,0.844828,0.847410,0.856835,0.844828
5,0.030200,0.696733,0.850575,0.852113,0.855741,0.850575



Test Set Results for answerdotai/ModernBERT-base
Accuracy: 0.8343
F1: 0.8339
Precision: 0.8337
Recall: 0.8343

Detailed Classification Report:
              precision    recall  f1-score   support

           L       0.87      0.88      0.87       115
           H       0.76      0.75      0.76        60

    accuracy                           0.83       175
   macro avg       0.82      0.81      0.82       175
weighted avg       0.83      0.83      0.83       175


Testing combination: target_with_position
Columns: ['Target clause', 'Number of the target clause in a sentence']

Training answerdotai/ModernBERT-base on full training set
Columns: ['Target clause', 'Number of the target clause in a sentence']


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.242000,0.339246,0.862069,0.864210,0.872158,0.862069
2,0.178800,0.379885,0.867816,0.870692,0.890171,0.867816
3,0.240200,0.366422,0.896552,0.896942,0.897585,0.896552


In [26]:
ablation_df = pd.DataFrame(ablation_results).T
ablation_df = ablation_df.reset_index().rename(columns={'index': 'combination'})

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
metrics_plot = ['accuracy', 'f1', 'precision', 'recall']

for idx, metric in enumerate(metrics_plot):
    ax = axes[idx // 2, idx % 2]

    x_pos = np.arange(len(ablation_df))
    values = ablation_df[metric].values

    bars = ax.bar(x_pos, values, alpha=0.7)

    # Color the bar with highest value
    max_idx = values.argmax()
    bars[max_idx].set_color('green')
    bars[max_idx].set_alpha(0.9)

    ax.set_xlabel('Column Combination')
    ax.set_ylabel(metric.capitalize())
    ax.set_title(f'{metric.capitalize()} by Column Combination')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(ablation_df['combination'], rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for i, v in enumerate(values):
        ax.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('ablation_column_combinations.png', dpi=300, bbox_inches='tight')
plt.show()

NameError: name 'ablation_results' is not defined

In [ ]:
print("\n" + "="*80)
print("ABLATION STUDY SUMMARY: COLUMN COMBINATIONS")
print("="*80)
print(ablation_df.to_string(index=False))

# Find best combination for each metric
print("\nBest combinations per metric:")
for metric in metrics_plot:
    best_combo = ablation_df.loc[ablation_df[metric].idxmax(), 'combination']
    best_value = ablation_df[metric].max()
    print(f"{metric.capitalize()}: {best_combo} ({best_value:.4f})")

In [ ]:
print("\n" + "="*80)
print("ABLATION STUDY: LAYER FREEZING")
print("="*80)

def train_with_frozen_layers(model_name, train_dataset, val_dataset, test_dataset,
                             columns, n_layers_to_freeze=0):
    """Train model with specified number of frozen layers"""
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Prepare datasets
    train_data = prepare_dataset(train_dataset, columns, tokenizer)
    val_data = prepare_dataset(val_dataset, columns, tokenizer)
    test_data = prepare_dataset(test_dataset, columns, tokenizer)

    train_dataset_obj = TextDataset(train_data)
    val_dataset_obj = TextDataset(val_data)
    test_dataset_obj = TextDataset(test_data)

    # Initialize model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )

    # Freeze layers
    if n_layers_to_freeze > 0:
        # For BERT/RoBERTa, encoder layers are in model.bert.encoder.layer or model.roberta.encoder.layer
        encoder_attr = 'bert' if 'bert' in model_name else 'roberta'
        if hasattr(model, encoder_attr):
            encoder = getattr(model, encoder_attr)
            for i in range(n_layers_to_freeze):
                for param in encoder.encoder.layer[i].parameters():
                    param.requires_grad = False
            print(f"Frozen first {n_layers_to_freeze} layers")

    # Training arguments (fewer epochs for ablation)
    training_args = TrainingArguments(
        output_dir=f'./results_frozen_{n_layers_to_freeze}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        logging_dir='./logs',
        logging_steps=10,
        seed=42
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_obj,
        eval_dataset=val_dataset_obj,
        compute_metrics=compute_metrics
    )

    trainer.train()
    test_results = trainer.evaluate(test_dataset_obj)

    return test_results

In [ ]:
freeze_configs = [0, 3, 6]  # Number of layers to freeze
freezing_results = {}

In [27]:
for n_freeze in freeze_configs:
    print(f"\nTesting with {n_freeze} frozen layers...")
    try:
        results = train_with_frozen_layers(
            best_model_name,
            dataset['train'],
            dataset['dev'],
            dataset['test'],
            selected_columns,
            n_freeze
        )

        freezing_results[n_freeze] = {
            'accuracy': results['eval_accuracy'],
            'f1': results['eval_f1'],
            'precision': results['eval_precision'],
            'recall': results['eval_recall']
        }

        print(f"Results with {n_freeze} frozen layers:")
        print(f"  Accuracy: {results['eval_accuracy']:.4f}")
        print(f"  F1: {results['eval_f1']:.4f}")
    except Exception as e:
        print(f"Error with {n_freeze} frozen layers: {str(e)}")
        continue

NameError: name 'freeze_configs' is not defined

In [ ]:
freezing_df = pd.DataFrame(freezing_results).T
freezing_df = freezing_df.reset_index().rename(columns={'index': 'frozen_layers'})
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: All metrics
ax1 = axes[0]
x_pos = np.arange(len(freezing_df))
width = 0.2

for i, metric in enumerate(['accuracy', 'f1', 'precision', 'recall']):
    ax1.bar(x_pos + i * width, freezing_df[metric], width,
            label=metric.capitalize(), alpha=0.8)

ax1.set_xlabel('Number of Frozen Layers')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance vs Layer Freezing')
ax1.set_xticks(x_pos + width * 1.5)
ax1.set_xticklabels(freezing_df['frozen_layers'].astype(int))
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: F1 score focus
ax2 = axes[1]
ax2.plot(freezing_df['frozen_layers'], freezing_df['f1'], marker='o',
         linewidth=2, markersize=8, color='blue')
ax2.set_xlabel('Number of Frozen Layers')
ax2.set_ylabel('F1 Score')
ax2.set_title('F1 Score vs Layer Freezing')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('ablation_layer_freezing.png', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
# Cell 22: Print layer freezing summary
print("\n" + "="*80)
print("ABLATION STUDY SUMMARY: LAYER FREEZING")
print("="*80)
print(freezing_df.to_string(index=False))

print("\nBest configuration:")
best_freeze = freezing_df.loc[freezing_df['f1'].idxmax(), 'frozen_layers']
best_f1 = freezing_df['f1'].max()
print(f"Frozen layers: {int(best_freeze)}, F1 score: {best_f1:.4f}")

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY AND RECOMMENDATIONS")
print("="*80)

print("\n1. BEST MODEL (from cross-validation):")
print(f"   Model: {best_model_name}")
print(f"   Mean F1: {cv_results[best_model_name]['stats']['f1']['mean']:.4f}")
print(f"   Std: {cv_results[best_model_name]['stats']['f1']['std']:.4f}")
print(f"   95% CI: [{cv_results[best_model_name]['stats']['f1']['ci_lower']:.4f}, "
      f"{cv_results[best_model_name]['stats']['f1']['ci_upper']:.4f}]")

print("\n2. BEST COLUMN COMBINATION:")
best_combo = ablation_df.loc[ablation_df['f1'].idxmax(), 'combination']
best_combo_f1 = ablation_df['f1'].max()
best_combo_cols = column_combinations[best_combo]
print(f"   Combination: {best_combo}")
print(f"   Columns: {best_combo_cols}")
print(f"   F1 score: {best_combo_f1:.4f}")

print("\n3. OPTIMAL LAYER FREEZING:")
print(f"   Frozen layers: {int(best_freeze)}")
print(f"   F1 score: {best_f1:.4f}")

print("\n4. KEY INSIGHTS:")
# Compare target_only vs all_columns
if 'target_only' in ablation_results and 'all_columns' in ablation_results:
    target_only_f1 = ablation_results['target_only']['f1']
    all_cols_f1 = ablation_results['all_columns']['f1']
    improvement = ((all_cols_f1 - target_only_f1) / target_only_f1) * 100

    print(f"   - Adding contextual columns improved F1 by {improvement:.2f}%")
    print(f"     (from {target_only_f1:.4f} to {all_cols_f1:.4f})")

# Compare full fine-tuning vs freezing
if 0 in freezing_results and len(freezing_results) > 1:
    full_finetune_f1 = freezing_results[0]['f1']
    frozen_f1_values = [v['f1'] for k, v in freezing_results.items() if k > 0]
    if frozen_f1_values:
        best_frozen_f1 = max(frozen_f1_values)
        if best_frozen_f1 >= full_finetune_f1 * 0.98:  # Within 2%
            print(f"   - Layer freezing achieves comparable performance")
            print(f"     (reduces training time without significant F1 loss)")

print("\n" + "="*80)
print("Analysis complete! All results saved to PNG files.")
print("="*80)

In [ ]:
# Train final model with best configuration
print("\nTraining final model with optimal configuration...")

final_model_results = train_and_evaluate_best_model(
    best_model_name,
    dataset['train'],
    dataset['dev'],
    dataset['test'],
    best_combo_cols
)

In [ ]:
save_directory = './best_model_for_inference'

print(f"\nSaving model to: {save_directory}")
final_model_results['trainer'].save_model(save_directory)

# Also save the tokenizer
tokenizer = AutoTokenizer.from_pretrained(best_model_name)
tokenizer.save_pretrained(save_directory)

print(f"✓ Model saved successfully!")
print(f"✓ Tokenizer saved successfully!")

# Save configuration info
import json

config_info = {
    'model_name': best_model_name,
    'best_columns': best_combo_cols,
    'column_combination_name': best_combo,
    'test_accuracy': float(final_model_results['test_results']['eval_accuracy']),
    'test_f1': float(final_model_results['test_results']['eval_f1']),
    'test_precision': float(final_model_results['test_results']['eval_precision']),
    'test_recall': float(final_model_results['test_results']['eval_recall']),
    'label_mapping': {'0': 'L', '1': 'H'}
}

with open(f'{save_directory}/config_info.json', 'w') as f:
    json.dump(config_info, f, indent=2)

print(f"✓ Configuration info saved!")

print("\n" + "="*80)
print("MODEL READY FOR INFERENCE")
print("="*80)
print(f"Location: {save_directory}")
print(f"Test F1 Score: {config_info['test_f1']:.4f}")
print(f"Columns to use: {best_combo_cols}")

In [ ]:
def load_model_for_inference(model_path='./best_model_for_inference'):
    """Load saved model and tokenizer for inference"""
    import json
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import torch

    # Load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Load config info
    with open(f'{model_path}/config_info.json', 'r') as f:
        config = json.load(f)

    # Set model to evaluation mode
    model.eval()

    print(f"Model loaded from: {model_path}")
    print(f"Expected columns: {config['best_columns']}")
    print(f"Label mapping: {config['label_mapping']}")

    return model, tokenizer, config

def predict_single_example(model, tokenizer, config, example_dict):
    """
    Predict label for a single example

    Args:
        model: Loaded model
        tokenizer: Loaded tokenizer
        config: Configuration dict with column info
        example_dict: Dictionary with keys matching dataset columns

    Returns:
        predicted_label: 'H' or 'L'
        confidence: Probability of predicted class
    """
    import torch

    # Concatenate columns as specified in config
    texts = []
    for col in config['best_columns']:
        if col in example_dict and example_dict[col] and str(example_dict[col]).strip():
            texts.append(str(example_dict[col]).strip())

    input_text = ' [SEP] '.join(texts)

    # Tokenize
    inputs = tokenizer(
        input_text,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][predicted_class].item()

    # Map to label
    predicted_label = config['label_mapping'][str(predicted_class)]

    return predicted_label, confidence

def predict_batch(model, tokenizer, config, examples_list):
    """
    Predict labels for a batch of examples

    Args:
        model: Loaded model
        tokenizer: Loaded tokenizer
        config: Configuration dict with column info
        examples_list: List of dictionaries with keys matching dataset columns

    Returns:
        predictions: List of tuples (predicted_label, confidence)
    """
    import torch

    # Prepare texts
    input_texts = []
    for example in examples_list:
        texts = []
        for col in config['best_columns']:
            if col in example and example[col] and str(example[col]).strip():
                texts.append(str(example[col]).strip())
        input_texts.append(' [SEP] '.join(texts))

    # Tokenize batch
    inputs = tokenizer(
        input_texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_classes = torch.argmax(probabilities, dim=-1).cpu().numpy()
        confidences = probabilities.max(dim=-1).values.cpu().numpy()

    # Map to labels
    predictions = []
    for pred_class, conf in zip(predicted_classes, confidences):
        label = config['label_mapping'][str(pred_class)]
        predictions.append((label, float(conf)))

    return predictions

print("✓ Inference functions created!")
print("\nHelper functions available:")
print("  - load_model_for_inference(): Load saved model")
print("  - predict_single_example(): Predict one example")
print("  - predict_batch(): Predict multiple examples")

In [ ]:
# Cell 26: Test inference on sample data
print("\n" + "="*80)
print("TESTING INFERENCE")
print("="*80)

# Load the model
loaded_model, loaded_tokenizer, loaded_config = load_model_for_inference()

# Test on a few examples from test set
print("\nTesting on sample examples from test set:")
print("-" * 80)

for i in range(min(5, len(dataset['test']))):
    example = dataset['test'][i]

    # Get prediction
    pred_label, confidence = predict_single_example(
        loaded_model,
        loaded_tokenizer,
        loaded_config,
        example
    )

    true_label = example['Label']

    print(f"\nExample {i+1}:")
    print(f"  Target sentence: {example['Sentence'][:100]}...")
    print(f"  True label: {true_label}")
    print(f"  Predicted label: {pred_label}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if pred_label == true_label else '✗'}")

In [ ]:
print("\n" + "="*80)
print("BATCH INFERENCE EXAMPLE")
print("="*80)

# Take 10 examples for batch prediction
batch_examples = [dataset['test'][i] for i in range(min(10, len(dataset['test'])))]

print(f"\nPredicting {len(batch_examples)} examples in batch...")

# Get predictions
batch_predictions = predict_batch(
    loaded_model,
    loaded_tokenizer,
    loaded_config,
    batch_examples
)

# Calculate accuracy
correct = sum(1 for (pred, _), ex in zip(batch_predictions, batch_examples)
              if pred == ex['Label'])
accuracy = correct / len(batch_examples)

print(f"\nBatch prediction complete!")
print(f"Accuracy on sample: {accuracy:.2%} ({correct}/{len(batch_examples)})")

# Show predictions
print("\nPredictions:")
print("-" * 80)
for i, ((pred_label, confidence), example) in enumerate(zip(batch_predictions, batch_examples)):
    true_label = example['Label']
    status = '✓' if pred_label == true_label else '✗'
    print(f"{i+1}. True: {true_label} | Pred: {pred_label} | Conf: {confidence:.3f} {status}")


In [28]:
inference_script = '''#!/usr/bin/env python3
"""
Standalone inference script for the trained text classifier
Usage: python inference.py
"""

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

def load_model(model_path='./best_model_for_inference'):
    """Load the trained model"""
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    with open(f'{model_path}/config_info.json', 'r') as f:
        config = json.load(f)

    model.eval()
    return model, tokenizer, config

def predict(model, tokenizer, config, example_dict):
    """Make a prediction"""
    # Concatenate required columns
    texts = []
    for col in config['best_columns']:
        if col in example_dict and example_dict[col]:
            texts.append(str(example_dict[col]).strip())

    input_text = ' [SEP] '.join(texts)

    # Tokenize and predict
    inputs = tokenizer(input_text, truncation=True, padding='max_length',
                      max_length=512, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        pred_class = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_class].item()

    label = config['label_mapping'][str(pred_class)]
    return label, confidence

if __name__ == '__main__':
    # Load model
    print("Loading model...")
    model, tokenizer, config = load_model()
    print(f"Model loaded! Required columns: {config['best_columns']}")

    # Example usage
    example = {
        'Target sentence': 'Your target sentence here',
        'Last argument': 'Your last argument here',
        'Thesis': 'Your thesis here',
        'Target sentence segment': 'Your segment here'
    }

    label, confidence = predict(model, tokenizer, config, example)
    print(f"\\nPrediction: {label} (confidence: {confidence:.4f})")
'''

# Save the inference script
with open('inference.py', 'w') as f:
    f.write(inference_script)

print("\n" + "="*80)
print("STANDALONE INFERENCE SCRIPT CREATED")
print("="*80)
print("✓ Script saved as: inference.py")


STANDALONE INFERENCE SCRIPT CREATED
✓ Script saved as: inference.py


In [29]:
import shutil

zip_filename = 'best_model_for_inference'
print(f"\nCreating zip file: {zip_filename}.zip")

shutil.make_archive(zip_filename, 'zip', './best_model_for_inference_dim2')


Creating zip file: best_model_for_inference.zip


FileNotFoundError: [Errno 2] No such file or directory: './best_model_for_inference_dim2'

In [ ]:
from google.colab import files
files.download("/content/best_model_for_inference_dim2.zip")